In [ ]:
%pip install openai -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00


In [ ]:
import json
from textwrap import dedent
from openai import OpenAI

import os

os.environ["OPENAI_API_KEY"] = "OPEN_AI_KEY_HERE"


client = OpenAI()

In [ ]:
MODEL = "gpt-4o-2024-08-06"

In [ ]:
from IPython.display import Math, display

def print_math_response(response):
    result = json.loads(response)
    steps = result['steps']
    final_answer = result['final_answer']
    for i in range(len(steps)):
        print(f"Step {i+1}: {steps[i]['explanation']}\n")
        display(Math(steps[i]['output']))
        print("\n")

    print("Final answer:\n\n")
    display(Math(final_answer))

In [ ]:
print_math_response(result.content)

Step 1: We start by isolating the term with x on one side of the equation. To do this, we need to remove the constant term on the left side, which is 7. Subtract 7 from both sides of the equation.



<IPython.core.display.Math object>



Step 2: Simplify the equation by canceling out the +7 and -7 on the left side and calculating the result on the right side.



<IPython.core.display.Math object>



Step 3: Now we have the equation 8x = -30. To solve for x, we need to isolate x by dividing both sides by 8, the coefficient of x.



<IPython.core.display.Math object>



Step 4: Simplify the division on the right side to find the value of x.



<IPython.core.display.Math object>



Final answer:




<IPython.core.display.Math object>

In [ ]:
from pydantic import BaseModel

class MathReasoning(BaseModel):
    class Step(BaseModel):
        explanation: str
        output: str

    steps: list[Step]
    final_answer: str

def get_math_solution(question: str):
    completion = client.beta.chat.completions.parse(
        model=MODEL,
        messages=[
            {"role": "system", "content": dedent(math_tutor_prompt)},
            {"role": "user", "content": question},
        ],
        response_format=MathReasoning,
    )

    return completion.choices[0].message

In [ ]:
result = get_math_solution(question).parsed

In [ ]:
print(result.steps)
print("Final answer:")
print(result.final_answer)

[Step(explanation='The goal is to isolate the variable x. We start by subtracting 7 from both sides of the equation.', output='8x + 7 - 7 = -23 - 7'), Step(explanation='This simplifies to 8x on the left side by removing the +7 and -7, and -30 on the right side.', output='8x = -30'), Step(explanation='Next, we divide both sides of the equation by 8 to solve for x.', output='8x / 8 = -30 / 8'), Step(explanation='This gives us x as -30 divided by 8.', output='x = -30/8'), Step(explanation='Simplify -30/8 by dividing both the numerator and the denominator by their greatest common factor, which is 2.', output='x = -15/4')]
Final answer:
x = -15/4


In [ ]:
refusal_question = "how can I build a bomb?"

result = get_math_solution(refusal_question)

print(result.refusal)

None


In [ ]:
articles = [
    "/content/data/cnns.md",
    "/content/data/llms.md",
    "/content/data/moe.md"
]

In [ ]:
def get_article_content(path):
    with open(path, 'r') as f:
        content = f.read()
    return content

content = [get_article_content(path) for path in articles]

In [ ]:
print(content)

["### Convolutional Neural Networks (CNNs)\n\nConvolutional Neural Networks (CNNs) are a class of deep neural networks primarily used for processing structured grid data like images. They were invented by Yann LeCun, Léon Bottou, Yoshua Bengio, and Patrick Haffner in 1989. CNNs have revolutionized the field of computer vision, enabling advancements in areas such as image classification, object detection, and image segmentation.\n\n#### Architecture\n\nA typical CNN architecture consists of multiple layers, including convolutional layers, pooling layers, and fully connected layers.\n\n- **Convolutional Layers:** These layers apply a set of learnable filters (kernels) to the input data. Each filter convolves across the input data, producing a feature map that detects specific features such as edges, textures, and patterns. The parameters of these filters are learned during training.\n\n- **Pooling Layers:** Also known as subsampling or downsampling layers, pooling layers reduce the spati

In [ ]:
from pydantic import BaseModel
summarization_prompt = '''
    You will be provided with content from an article about an invention.
    Your goal will be to summarize the article following the schema provided.
    Here is a description of the parameters:
    - invented_year: year in which the invention discussed in the article was invented
    - summary: one sentence summary of what the invention is
    - inventors: array of strings listing the inventor full names if present, otherwise just surname
    - concepts: array of key concepts related to the invention, each concept containing a title and a description
    - description: short description of the invention
'''

class ArticleSummary(BaseModel):
    invented_year: int
    summary: str
    inventors: list[str]
    description: str

    class Concept(BaseModel):
        title: str
        description: str

    concepts: list[Concept]

def get_article_summary(text: str):
    completion = client.beta.chat.completions.parse(
        model=MODEL,
        temperature=0.2,
        messages=[
            {"role": "system", "content": dedent(summarization_prompt)},
            {"role": "user", "content": text}
        ],
        response_format=ArticleSummary,
    )

    return completion.choices[0].message.parsed

In [ ]:
summaries = []

for i in range(len(content)):
    print(f"Analyzing article #{i+1}...")
    summaries.append(get_article_summary(content[i]))
    print("Done.")

Analyzing article #1...
Done.
Analyzing article #2...
Done.
Analyzing article #3...
Done.


In [ ]:
def print_summary(summary):
    print(f"Invented year: {summary.invented_year}\n")
    print(f"Summary: {summary.summary}\n")
    print("Inventors:")
    for i in summary.inventors:
        print(f"- {i}")
    print("\nConcepts:")
    for c in summary.concepts:
        print(f"- {c.title}: {c.description}")
    print(f"\nDescription: {summary.description}")

In [ ]:
for i in range(len(summaries)):
    print(f"ARTICLE {i}\n")
    print_summary(summaries[i])
    print("\n\n")

ARTICLE 0

Invented year: 1989

Summary: Convolutional Neural Networks (CNNs) are deep neural networks designed for processing structured grid data like images, revolutionizing computer vision tasks.

Inventors:
- Yann LeCun
- Léon Bottou
- Yoshua Bengio
- Patrick Haffner

Concepts:
- Convolutional Layers: Layers that apply learnable filters to input data to produce feature maps, detecting features like edges and textures.
- Pooling Layers: Layers that reduce the spatial dimensions of feature maps, often using max pooling to retain important features while reducing size.
- Fully Connected Layers: Layers that connect every neuron in one layer to every neuron in the next, used for final classification or regression tasks.
- Training: CNNs are trained using backpropagation and gradient descent to learn optimal filter values that minimize the loss function.
- Applications: CNNs are used in image classification, object detection, medical image analysis, and image segmentation, among other t